In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

In [2]:
def swiss_roll(N):
    dataset = np.ndarray((N, 3))
    phi = np.random.uniform(1.5 * np.pi, 4.5 * np.pi, N)

    dataset[:, 0] = phi * np.cos(phi)
    dataset[:, 1] = phi * np.sin(phi)
    dataset[:, 2] = np.random.uniform(0, 10, N)

    return dataset

In [3]:
X = swiss_roll(100)

In [4]:
# Step 1: Find the k nearest neighbors of each point
n_neighbors = 10
nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean').fit(X)

distances, indexes = nbrs.kneighbors(X)

In [5]:
# Step 2: Compute relationships between neighbors

def calculate_angle(A, B, C):
    BA = A - B
    BC = C - B

    dot_product = np.dot(BA, BC)
    norm_product = np.linalg.norm(BA) * np.linalg.norm(BC)
    cosine_angle = dot_product / norm_product
    angle = np.arccos(cosine_angle)

    return angle




In [6]:
# Given two points A and B, find the point C that is most collinear with A and B, chosen within a list
def most_collinear_point(point1, point2, set_of_points):
    max_angle = 0
    collinear_idx = -1
    for i in range(len(set_of_points)):
        if(np.array_equal(set_of_points[i, :], point1) or np.array_equal(set_of_points[i, :], point2)):
            # print("Skipping calculation for point1 or point2")
            continue
        else:
            angle = calculate_angle(point1, point2, set_of_points[i])

        if angle > max_angle:
            max_angle = angle
            collinear_idx = i
    
    return max_angle, collinear_idx

In [7]:
def compute_relationships_points(dataset, nn_indices):
    n_neighbors = nn_indices.shape[1]
    collinear_neighbors = np.empty((nn_indices.shape[0], n_neighbors - 1))
    angles = np.ones((nn_indices.shape[0], n_neighbors - 1)) * (-1)

    for i in range(nn_indices.shape[0]):

        # indexes of the nearest neighbors of the point A = dataset[i, :]
        nearest_neighbors_A = nn_indices[i, 1:]

        # Coordinates of the nearest neighbors of A
        B = dataset[nearest_neighbors_A, :]
    
        # For each neighbor of A
        for j in range(n_neighbors - 1):
            # Coordinates of the nearest neighbors of B
            nearest_neighbors_B = nn_indices[nearest_neighbors_A[j]]

            # Find the point C that is most collinear with A and B
            angle, collinear_idx = most_collinear_point(dataset[i, :], B[j, :], dataset[nearest_neighbors_B])
            angles[i, j] = angle

            # store index and angle of the most collinear point
            collinear_neighbors[i, j] = collinear_idx
    return collinear_neighbors, angles

coll_neigh, angles = compute_relationships_points(X, indexes)